# 🎨 MAP-Elites Creative Architecture Search - Demo

MAP-Elites is a **Quality-Diversity** algorithm that discovers diverse, high-quality architectures.

**Key advantages:**
- ✅ Guaranteed diversity across behavior space
- ✅ No neural networks in search (faster!)
- ✅ Simpler than Deep RL
- ✅ More reliable and predictable
- ✅ Systematic coverage of design space

**Estimated time:** 3-10 minutes

---

## 📦 Setup & Imports

In [ ]:
# Install if needed
# !pip install torch torchvision torch-geometric networkx scipy matplotlib tqdm

In [ ]:
import torch
import os
from datetime import datetime
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Using device: {device}")
if device == 'cpu':
    print("⚠️  No GPU detected. Recommend MNIST with fewer iterations.")

## ⚙️ Configuration

In [ ]:
# ===== MAP-ELITES CONFIGURATION =====

# Dataset: 'mnist', 'fashion', 'cifar10'
DATASET = 'mnist'

# Number of iterations
ITERATIONS = 200

# Behavior space resolution (depth_bins, width_bins, skip_bins)
# Low: (3, 3, 3) = 27 cells
# Medium: (5, 5, 4) = 100 cells
# High: (7, 7, 5) = 245 cells
DEPTH_BINS = 5
WIDTH_BINS = 5
SKIP_BINS = 4

# Number of top architectures to evaluate
TOP_K = 10

# Output directory
OUTPUT_DIR = f'demo_results_mapelites/{DATASET}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

print("📋 Configuration:")
print(f"  Algorithm: MAP-Elites (Quality-Diversity)")
print(f"  Dataset: {DATASET}")
print(f"  Iterations: {ITERATIONS}")
print(f"  Behavior Space: {DEPTH_BINS}×{WIDTH_BINS}×{SKIP_BINS} = {DEPTH_BINS*WIDTH_BINS*SKIP_BINS} cells")
print(f"  Top K: {TOP_K}")
print(f"  Output: {OUTPUT_DIR}")

# Time estimate
time_estimates = {
    'mnist': ITERATIONS * 0.015 + TOP_K * 0.15,
    'fashion': ITERATIONS * 0.02 + TOP_K * 0.2,
    'cifar10': ITERATIONS * 0.03 + TOP_K * 0.4
}
print(f"\n⏱️  Estimated time: ~{time_estimates[DATASET]:.1f} minutes")

## 🏗️ Load Components

In [ ]:
from map_elites import MAPElites, BehaviorSpace, MutationOperator, QualityDiversityArchive
from architecture import ArchitectureState, OPERATION_POOL
from evaluation import ConvNet, train_architecture
from utils import save_all_results, load_architecture_json
from visualize import visualize_architecture, create_results_report

print("✅ All modules loaded!")

## 🔧 Configure Dataset

In [ ]:
import evaluation
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

if DATASET == 'mnist':
    def get_mnist_loaders(batch_size=128, subset_size=None):
        transform = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
            transforms.Normalize((0.1307,)*3, (0.3081,)*3)
        ])
        
        trainset = torchvision.datasets.MNIST(
            root='./data', train=True, download=True, transform=transform
        )
        testset = torchvision.datasets.MNIST(
            root='./data', train=False, download=True, transform=transform
        )
        
        if subset_size:
            indices = np.random.choice(len(trainset), subset_size, replace=False)
            trainset = Subset(trainset, indices)
        
        return (DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2),
               DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2))
    
    evaluation.get_cifar10_loaders = get_mnist_loaders
    print("📊 Dataset: MNIST")

elif DATASET == 'fashion':
    def get_fashion_loaders(batch_size=128, subset_size=None):
        transform = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
            transforms.Normalize((0.2860,)*3, (0.3530,)*3)
        ])
        
        trainset = torchvision.datasets.FashionMNIST(
            root='./data', train=True, download=True, transform=transform
        )
        testset = torchvision.datasets.FashionMNIST(
            root='./data', train=False, download=True, transform=transform
        )
        
        if subset_size:
            indices = np.random.choice(len(trainset), subset_size, replace=False)
            trainset = Subset(trainset, indices)
        
        return (DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2),
               DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2))
    
    evaluation.get_cifar10_loaders = get_fashion_loaders
    print("📊 Dataset: Fashion-MNIST")

else:
    print("📊 Dataset: CIFAR-10")

## 🎯 Initialize MAP-Elites

In [ ]:
print("📦 Initializing MAP-Elites...")

# Define behavior space
behavior_space = BehaviorSpace(
    depth_bins=DEPTH_BINS,
    width_bins=WIDTH_BINS,
    skip_bins=SKIP_BINS
)

# Define mutation operator
mutation_operator = MutationOperator()

# Initialize MAP-Elites
map_elites = MAPElites(
    behavior_space=behavior_space,
    mutation_operator=mutation_operator,
    operation_strategy='diverse'
)

print(f"✅ MAP-Elites initialized")
print(f"   Total cells: {behavior_space.get_total_cells()}")
print(f"   Mutation types: {len(mutation_operator.mutation_types)}")

## 🚀 Run MAP-Elites

Watch the coverage increase as diverse architectures are discovered!

In [ ]:
# Define evaluation function
def evaluate_architecture(arch: ArchitectureState) -> float:
    """Quick evaluation during search"""
    try:
        accuracy = train_architecture(
            arch,
            epochs=3,
            device=device,
            subset_size=10000
        )
        return accuracy
    except Exception as e:
        print(f"Error: {e}")
        return 0.1

print(f"🎯 Running MAP-Elites for {ITERATIONS} iterations...\n")

archive = map_elites.run(
    num_iterations=ITERATIONS,
    evaluate_fn=evaluate_architecture,
    verbose=True
)

print("\n✅ MAP-Elites complete!")

## 📊 Results Summary

In [ ]:
stats = archive.get_stats()

print("📋 MAP-Elites Statistics")
print("="*60)
print(f"Coverage:        {stats['coverage']:.2%}")
print(f"Cells filled:    {stats['num_filled']}/{behavior_space.get_total_cells()}")
print(f"Best performance: {stats['best_performance']:.4f}")
print(f"Mean performance: {stats['mean_performance']:.4f}")
print(f"Total evaluated:  {stats['total_evaluated']}")
print("="*60)

## 📈 Visualize Behavior Space Coverage

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get all behaviors and performances
all_archs = archive.get_all_architectures()
behaviors = [b for _, _, b in all_archs]
performances = [p for _, p, _ in all_archs]

# Create 2D heatmap (depth vs width)
heatmap = np.zeros((DEPTH_BINS, WIDTH_BINS))
counts = np.zeros((DEPTH_BINS, WIDTH_BINS))

for (d, w, s), perf in zip(behaviors, performances):
    heatmap[d, w] += perf
    counts[d, w] += 1

# Average performance per cell
heatmap = np.divide(heatmap, counts, where=counts>0, out=np.zeros_like(heatmap))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Coverage heatmap
im1 = axes[0].imshow((counts > 0).astype(int), cmap='Blues', aspect='auto')
axes[0].set_title('Coverage (Depth vs Width)')
axes[0].set_xlabel('Width Bin')
axes[0].set_ylabel('Depth Bin')
plt.colorbar(im1, ax=axes[0], label='Filled')

# Performance heatmap
im2 = axes[1].imshow(heatmap, cmap='viridis', aspect='auto')
axes[1].set_title('Average Performance (Depth vs Width)')
axes[1].set_xlabel('Width Bin')
axes[1].set_ylabel('Depth Bin')
plt.colorbar(im2, ax=axes[1], label='Accuracy')

plt.tight_layout()
plt.show()

print(f"\n📊 Coverage: {stats['coverage']:.2%} of behavior space explored")

## 🎓 Final Evaluation

Fully train the top architectures.

In [ ]:
print(f"Evaluating top {TOP_K} architectures...\n")

all_archs = archive.get_all_architectures()
all_archs.sort(key=lambda x: x[1], reverse=True)

final_results = []

for i, (arch, search_perf, behavior) in enumerate(all_archs[:TOP_K]):
    print(f"[{i+1}/{TOP_K}] Behavior: depth={behavior[0]}, width={behavior[1]}, skip={behavior[2]}")
    
    final_acc = train_architecture(
        arch,
        epochs=10,
        device=device,
        subset_size=None
    )
    
    print(f"    ✅ Accuracy: {final_acc:.4f}\n")
    
    final_results.append({
        'architecture': arch,
        'search_reward': search_perf,
        'final_accuracy': final_acc,
        'trajectory': [],
        'behavior': behavior
    })

print("✅ Evaluation complete!")

## 💾 Save Results

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Prepare data for saving
best_archs = [{
    'architecture': item['architecture'],
    'reward': item['performance'],
    'trajectory': [],
    'episode': 0
} for item in archive.history]

dummy_stats = {
    'episode_rewards': [item['performance'] for item in archive.history],
    'losses': [],
    'epsilons': []
}

print("💾 Saving results...\n")
save_all_results(OUTPUT_DIR, best_archs, final_results, dummy_stats, DATASET)

print("\n📊 Creating visualizations...")
create_results_report(OUTPUT_DIR)

print(f"\n✅ All results saved to: {OUTPUT_DIR}")

## 🏆 Best Architecture

In [ ]:
best = max(final_results, key=lambda x: x['final_accuracy'])
best_arch = best['architecture']

print("🏆 BEST ARCHITECTURE")
print("="*60)
print(f"Accuracy:  {best['final_accuracy']:.4f}")
print(f"Behavior:  {best['behavior']}")
print(f"Nodes:     {len(best_arch.nodes)}")
print(f"Edges:     {len(best_arch.edges)}")
print(f"Depth:     {best_arch.depth}")
print(f"Avg Width: {best_arch.avg_width:.1f}")
print(f"Parameters: {best_arch.total_params:,}")
print("="*60)

# Visualize
viz_path = os.path.join(OUTPUT_DIR, 'best_architecture.png')
visualize_architecture(best_arch, save_path=viz_path,
                      title=f"Best: Acc={best['final_accuracy']:.4f}, Behavior={best['behavior']}")
display(Image(filename=viz_path))

## 📊 Top 10 Results

In [ ]:
import pandas as pd

results_data = []
for i, result in enumerate(sorted(final_results, key=lambda x: x['final_accuracy'], reverse=True)[:10]):
    arch = result['architecture']
    behavior = result['behavior']
    results_data.append({
        'Rank': i+1,
        'Accuracy': f"{result['final_accuracy']:.4f}",
        'Behavior (D,W,S)': f"({behavior[0]},{behavior[1]},{behavior[2]})",
        'Nodes': len(arch.nodes),
        'Depth': arch.depth,
        'Avg Width': f"{arch.avg_width:.1f}",
        'Parameters': f"{arch.total_params:,}"
    })

df = pd.DataFrame(results_data)
display(df)

## 🎨 Diversity Showcase

Show architectures from different behavior cells.

In [ ]:
# Select diverse architectures
diverse_archs = []
seen_behaviors = set()

for arch, perf, behavior in sorted(all_archs, key=lambda x: x[1], reverse=True):
    if behavior not in seen_behaviors:
        diverse_archs.append((arch, perf, behavior))
        seen_behaviors.add(behavior)
    
    if len(diverse_archs) >= 5:
        break

print("🎨 Diverse Architectures from Different Cells:\n")

for i, (arch, perf, behavior) in enumerate(diverse_archs):
    print(f"Architecture {i+1}:")
    print(f"  Behavior: depth={behavior[0]}, width={behavior[1]}, skip={behavior[2]}")
    print(f"  Performance: {perf:.4f}")
    print(f"  Depth: {arch.depth}, Width: {arch.avg_width:.1f}, Nodes: {len(arch.nodes)}")
    print()

## ✅ Demo Complete!

### What you got:
- ✅ Diverse architecture portfolio
- ✅ Coverage across behavior space
- ✅ Models saved as .pth files
- ✅ Architectures saved as .json files
- ✅ Interactive HTML report

### Key advantages of MAP-Elites:
1. **Guaranteed diversity** - Systematic coverage
2. **No collapse** - Can't converge to single solution
3. **Simpler** - No neural networks in search
4. **Faster** - Less computation per iteration
5. **More reliable** - Deterministic behavior

**Open `report.html` to explore all results!** 🎉